In [52]:
import pandas as pd
import numpy as np

In [2]:
from ift6758.data import load_events_dataframe

# You can pass a season number (first year) as argument to load only one season
df = load_events_dataframe(2020)
df.head()


Found 57734 events


,game_id,season,game_type,game_date,venue,venue_location,away_team_id,away_team_abbrev,away_team_name,home_team_id,...,scoring_player_team_id,scoring_player_position_code,assist1_player_id,assist1_player_name,assist1_player_team_id,assist1_player_position_code,assist2_player_id,assist2_player_name,assist2_player_team_id,assist2_player_position_code
0,2020020001,20202021,2,2021-01-13,Wells Fargo Center,Philadelphia,5,PIT,Penguins,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020020001,20202021,2,2021-01-13,Wells Fargo Center,Philadelphia,5,PIT,Penguins,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020020001,20202021,2,2021-01-13,Wells Fargo Center,Philadelphia,5,PIT,Penguins,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020020001,20202021,2,2021-01-13,Wells Fargo Center,Philadelphia,5,PIT,Penguins,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020020001,20202021,2,2021-01-13,Wells Fargo Center,Philadelphia,5,PIT,Penguins,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# differente valeur de situation_code
df['situation_code'].unique()
# avoir le type de la colonne situation_code
df['situation_code'].dtype

dtype('O')

In [9]:
print(df.iloc[0])

game_id                                                              2020020001
season                                                                 20202021
game_type                                                                     2
game_date                                                            2021-01-13
venue                                                        Wells Fargo Center
venue_location                                                     Philadelphia
away_team_id                                                                  5
away_team_abbrev                                                            PIT
away_team_name                                                         Penguins
home_team_id                                                                  4
home_team_abbrev                                                            PHI
home_team_name                                                           Flyers
event_id                                

In [59]:
# Create a copy of the selected columns from the original DataFrame
dftest = df[['time_in_period', 'time_remaining', 'period_number', 
              'event_owner_team_id', 'home_team_id', 'home_team_name', 
              'away_team_id', 'away_team_name', 'type_desc_key', 'x_coord', 
              'y_coord', 'shooting_player_name', 'goalie_in_net_name', 
              'shot_type', 'goalie_in_net_position_code', 'situation_code']].copy()

# Create a new column 'is_empty_net' based on whether 'goalie_in_net_position_code' is NaN
dftest['is_empty_net'] = dftest['goalie_in_net_position_code'].isna().replace({True: 'yes', False: 'no'})

# Drop the 'goalie_in_net_position_code' column as it's no longer needed
dftest = dftest.drop(columns=['goalie_in_net_position_code'])

# Add 'event_team_name' column based on the owner team
dftest.loc[:, 'event_team_name'] = dftest.apply(
    lambda x: x['home_team_name'] if x['event_owner_team_id'] == x['home_team_id'] else x['away_team_name'],
    axis=1
)

# Define a function to get home and away player counts based on situation_code
def get_player_counts(situation_code):
    away_players = int(situation_code[0]) + int(situation_code[1])  # Adding the first two digits
    home_players = int(situation_code[2]) + int(situation_code[3])  # Adding the last two digits
    return away_players, home_players

# Create new columns for away_player and home_player
dftest[['away_player', 'home_player']] = dftest['situation_code'].astype(str).apply(get_player_counts).apply(pd.Series)

# Create a new column to determine the situation (even strength, power play, or shorthanded)
def determine_situation(row):
    if row['away_player'] == row['home_player']:
        return 'even strength'
    elif row['away_player'] > row['home_player'] and row['away_team_name'] == row['event_team_name']:
        return 'power play'
    elif row['away_player'] > row['home_player'] and row['away_team_name'] != row['event_team_name']:
        return 'shorthanded'
    elif row['home_player'] > row['away_player'] and row['home_team_name'] == row['event_team_name']:
        return 'power play'
    elif row['home_player'] > row['away_player'] and row['home_team_name'] != row['event_team_name']:
        return 'shorthanded'
    else:
        return 'unknown'

# Apply the function to create the new situation column
dftest['situation'] = dftest.apply(determine_situation, axis=1)

# Drop unnecessary columns
dftest = dftest.drop(columns=['away_team_name', 
                              'event_owner_team_id', 'home_team_id', 'away_team_id'])

# Display the final DataFrame
dftest


,time_in_period,time_remaining,period_number,home_team_name,type_desc_key,x_coord,y_coord,shooting_player_name,goalie_in_net_name,shot_type,situation_code,is_empty_net,event_team_name,away_player,home_player,situation
0,00:16,19:44,1,Flyers,shot-on-goal,-74,29,Travis Konecny,Tristan Jarry,wrist,1551,no,Flyers,6,6,even strength
1,00:34,19:26,1,Flyers,shot-on-goal,49,-25,Evan Rodrigues,Carter Hart,wrist,1551,no,Penguins,6,6,even strength
2,01:05,18:55,1,Flyers,shot-on-goal,-52,-31,Joel Farabee,Tristan Jarry,backhand,1551,no,Flyers,6,6,even strength
3,02:51,17:09,1,Flyers,shot-on-goal,43,39,Evan Rodrigues,Carter Hart,snap,1551,no,Penguins,6,6,even strength
4,03:44,16:16,1,Flyers,shot-on-goal,-53,8,Claude Giroux,Tristan Jarry,slap,1551,no,Flyers,6,6,even strength
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57729,16:19,03:41,3,Lightning,shot-on-goal,57,30,Blake Coleman,Carey Price,wrist,1551,no,Lightning,6,6,even strength
57730,17:13,02:47,3,Lightning,shot-on-goal,56,-31,Mikhail Sergachev,Carey Price,backhand,1551,no,Lightning,6,6,even strength
57731,19:01,00:59,3,Lightning,shot-on-goal,-45,-14,Tyler Toffoli,Andrei Vasilevskiy,wrist,0651,no,Canadiens,6,6,even strength
57732,19:10,00:50,3,Lightning,shot-on-goal,-77,-3,Cole Caufield,Andrei Vasilevskiy,backhand,0651,no,Canadiens,6,6,even strength
